In [1]:
import pandas as pd

c:\Users\cxh83\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\cxh83\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('building_data/trans.csv')
df

,POI_YM,POI_ID,POI_LON,POI_LAT,CATEGORY,POI_X,POI_Y
0,201901,420104,121.525828,25.090465,公車站,303038.626639,2.775901e+06
1,201901,416060,121.530708,25.104529,公車站,303524.745453,2.777461e+06
2,201901,415144,121.558438,25.147261,公車站,306301.930893,2.782205e+06
3,201901,413824,121.526556,25.092546,自行車租賃站,303111.160705,2.776132e+06
4,201901,418251,121.518204,25.089085,公車站,302270.190217,2.775745e+06
...,...,...,...,...,...,...,...
17430,202401,465686,121.567410,25.024740,自行車租賃站,307263.494018,2.768638e+06
17431,202401,473587,121.574507,25.045455,公車站,307970.008282,2.770935e+06
17432,202401,478512,121.557888,25.025944,公車站,306301.956644,2.768767e+06
17433,202401,477360,121.571998,25.044926,公車站,307717.082547,2.770876e+06


In [3]:
print(df['CATEGORY'].unique())

['公車站' '自行車租賃站' '公路設施' '捷運設施' '客運站' '鐵路設施' '轉運站']


In [4]:
trans1 = df[df['CATEGORY'].isin(['公車站', '自行車租賃站','捷運設施'])]
trans2 = df[df['CATEGORY'].isin(['公路設施', '鐵路設施', '轉運站', '客運站'])]

In [17]:
p_taipei = pd.read_csv('points_taipei.csv')
p_taipei['index'] = range(0, len(p_taipei))
p_taipei

,x1,y1,x2,y2,cx,cy,index
0,296103,2777535,296203,2777635,296153,2777585,0
1,296103,2777635,296203,2777735,296153,2777685,1
2,296103,2777735,296203,2777835,296153,2777785,2
3,296103,2777835,296203,2777935,296153,2777885,3
4,296103,2777935,296203,2778035,296153,2777985,4
...,...,...,...,...,...,...,...
27656,317103,2768935,317203,2769035,317153,2768985,27656
27657,317103,2769035,317203,2769135,317153,2769085,27657
27658,317103,2769135,317203,2769235,317153,2769185,27658
27659,317103,2769235,317203,2769335,317153,2769285,27659


In [18]:
p_new_taipei = pd.read_csv('points_new_taipei.csv')
p_new_taipei['index'] = range(0, len(p_new_taipei))
p_new_taipei

,x1,y1,x2,y2,cx,cy,index
0,278508,2777870,278608,2777970,278558,2777920,0
1,278508,2777970,278608,2778070,278558,2778020,1
2,278508,2778070,278608,2778170,278558,2778120,2
3,278508,2778170,278608,2778270,278558,2778220,3
4,278508,2778370,278608,2778470,278558,2778420,4
...,...,...,...,...,...,...,...
209743,351508,2767770,351608,2767870,351558,2767820,209743
209744,351608,2766970,351708,2767070,351658,2767020,209744
209745,351608,2767070,351708,2767170,351658,2767120,209745
209746,351608,2767270,351708,2767370,351658,2767320,209746


In [7]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

In [8]:
# 創建一個包含醫院位置的GeoDataFrame
trans_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(trans1['POI_X'], trans1['POI_Y'])])

trans1_100 = trans_points1.buffer(100)
trans1_500 = trans_points1.buffer(500)
trans1_800 = trans_points1.buffer(800)

trans1_100_gdf = gpd.GeoDataFrame(geometry=trans1_100, crs=trans_points1.crs)
trans1_500_gdf = gpd.GeoDataFrame(geometry=trans1_500, crs=trans_points1.crs)
trans1_800_gdf = gpd.GeoDataFrame(geometry=trans1_800, crs=trans_points1.crs)

In [9]:
# 創建一個包含醫院位置的GeoDataFrame2
trans_points2 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(trans2['POI_X'], trans2['POI_Y'])])

trans2_100 = trans_points2.buffer(100)
trans2_500 = trans_points2.buffer(500)
trans2_1000 = trans_points2.buffer(1000)

trans2_100_gdf = gpd.GeoDataFrame(geometry=trans2_100, crs=trans_points2.crs)
trans2_500_gdf = gpd.GeoDataFrame(geometry=trans2_500, crs=trans_points2.crs)
trans2_1000_gdf = gpd.GeoDataFrame(geometry=trans2_1000, crs=trans_points2.crs)

In [10]:
# 將百米方格加載為GeoDataFrame

def create_geometry_gdf(df):
    geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, x2, y1, y2 in zip(df['x1'], df['x2'], df['y1'], df['y2'])]
    geometry_gdf = gpd.GeoDataFrame(geometry=geometry)
    geometry_gdf['index'] = range(0, len(df))
    return geometry_gdf

geometry_gdf_t = create_geometry_gdf(p_taipei)
geometry_gdf_nt = create_geometry_gdf(p_new_taipei)
geometry_gdf_t

,geometry,index
0,"POLYGON ((296103.000 2777535.000, 296203.000 2...",0
1,"POLYGON ((296103.000 2777635.000, 296203.000 2...",1
2,"POLYGON ((296103.000 2777735.000, 296203.000 2...",2
3,"POLYGON ((296103.000 2777835.000, 296203.000 2...",3
4,"POLYGON ((296103.000 2777935.000, 296203.000 2...",4
...,...,...
27656,"POLYGON ((317103.000 2768935.000, 317203.000 2...",27656
27657,"POLYGON ((317103.000 2769035.000, 317203.000 2...",27657
27658,"POLYGON ((317103.000 2769135.000, 317203.000 2...",27658
27659,"POLYGON ((317103.000 2769235.000, 317203.000 2...",27659


In [11]:
def overlay_and_count(geometry_gdf, trans_gdf, prefix):
    intersection = geometry_gdf.overlay(trans_gdf, how='intersection')
    count_df = intersection.groupby('index').size().reset_index(name=prefix + '_cnt')
    return count_df

def calculate_trans_counts(geometry_gdf, trans1_100_gdf, trans1_500_gdf, trans1_800_gdf, trans2_100_gdf, trans2_500_gdf,trans2_1000_gdf):
    trans1_100_cnt = overlay_and_count(geometry_gdf, trans1_100_gdf, 'trans1_100')
    trans1_500_cnt = overlay_and_count(geometry_gdf, trans1_500_gdf, 'trans1_500')
    trans1_800_cnt = overlay_and_count(geometry_gdf, trans1_800_gdf, 'trans1_800')
    trans2_100_cnt = overlay_and_count(geometry_gdf, trans2_100_gdf, 'trans2_100')
    trans2_500_cnt = overlay_and_count(geometry_gdf, trans2_500_gdf, 'trans2_500')
    trans2_1000_cnt = overlay_and_count(geometry_gdf, trans2_1000_gdf, 'trans2_1000')
    return trans1_100_cnt, trans1_500_cnt, trans1_800_cnt, trans2_100_cnt, trans2_500_cnt,trans2_1000_cnt

# 呼叫函式
trans1_100_cnt_t, trans1_500_cnt_t, trans1_800_cnt_t, trans2_100_cnt_t, trans2_500_cnt_t, trans2_1000_cnt_t = calculate_trans_counts(geometry_gdf_t, trans1_100_gdf, trans1_500_gdf, trans1_800_gdf, trans2_100_gdf, trans2_500_gdf, trans2_1000_gdf)
trans1_100_cnt_nt, trans1_500_cnt_nt, trans1_800_cnt_nt, trans2_100_cnt_nt, trans2_500_cnt_nt, trans2_1000_cnt_nt = calculate_trans_counts(geometry_gdf_nt, trans1_100_gdf, trans1_500_gdf, trans1_800_gdf, trans2_100_gdf, trans2_500_gdf, trans2_1000_gdf)

In [19]:
def calculate_weighted_score(df, *args):
    # 進行多個 merge 操作
    for arg in args:
        df = df.merge(arg, on='index', how='left')
    
    # 將 NaN 值填充為 0
    df = df.fillna(0)
    
    # 計算加權分數
    df['交通加權分數'] = (
        df['trans1_100_cnt'] * 1 +
        df['trans1_500_cnt'] * 0.6 +
        df['trans1_800_cnt'] * 0.3 +
        df['trans2_100_cnt'] * 0.8 +
        df['trans2_500_cnt'] * 0.5 +
        df['trans2_1000_cnt'] * 0.2 
    )
    
    return df

# 呼叫函式
result_t = calculate_weighted_score(p_taipei, trans1_100_cnt_t, trans1_500_cnt_t, trans1_800_cnt_t, trans2_100_cnt_t, trans2_500_cnt_t, trans2_1000_cnt_t)
result_nt = calculate_weighted_score(p_new_taipei, trans1_100_cnt_nt, trans1_500_cnt_nt, trans1_800_cnt_nt, trans2_100_cnt_nt,  trans2_500_cnt_nt, trans2_1000_cnt_nt)



In [20]:
result_t.sort_values('交通加權分數', ascending=False)

,x1,y1,x2,y2,cx,cy,index,trans1_100_cnt,trans1_500_cnt,trans1_800_cnt,trans2_100_cnt,trans2_500_cnt,trans2_1000_cnt,交通加權分數
4061,301303,2770435,301403,2770535,301353,2770485,4061,24.0,86.0,185.0,0.0,0.0,4.0,131.9
2847,300503,2769635,300603,2769735,300553,2769685,2847,14.0,98.0,174.0,4.0,4.0,4.0,131.0
3893,301203,2770335,301303,2770435,301253,2770385,3893,19.0,91.0,185.0,0.0,0.0,4.0,129.9
2982,300603,2769535,300703,2769635,300653,2769585,2982,8.0,105.0,181.0,2.0,4.0,4.0,129.7
3262,300803,2769635,300903,2769735,300853,2769685,3262,6.0,101.0,199.0,0.0,4.0,4.0,129.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3236,300703,2784635,300803,2784735,300753,2784685,3236,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3235,300703,2784535,300803,2784635,300753,2784585,3235,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3234,300703,2784435,300803,2784535,300753,2784485,3234,0.0,0.0,0.0,0.0,0.0,0.0,0.0
835,298403,2779535,298503,2779635,298453,2779585,835,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
result_nt.sort_values('交通加權分數', ascending=False)

,x1,y1,x2,y2,cx,cy,index,trans1_100_cnt,trans1_500_cnt,trans1_800_cnt,trans2_100_cnt,trans2_500_cnt,trans2_1000_cnt,交通加權分數
9887,287308,2773870,287408,2773970,287358,2773920,9887,13.0,90.0,170.0,0.0,0.0,4.0,118.8
8439,286708,2773670,286808,2773770,286758,2773720,8439,11.0,95.0,162.0,0.0,0.0,6.0,117.8
9888,287308,2773970,287408,2774070,287358,2774020,9888,11.0,89.0,174.0,0.0,0.0,4.0,117.4
11130,287808,2774270,287908,2774370,287858,2774320,11130,13.0,85.0,176.0,0.0,0.0,2.0,117.2
9165,287008,2773770,287108,2773870,287058,2773820,9165,10.0,96.0,158.0,0.0,0.0,7.0,116.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142714,316708,2751270,316808,2751370,316758,2751320,142714,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142713,316708,2751170,316808,2751270,316758,2751220,142713,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142712,316708,2751070,316808,2751170,316758,2751120,142712,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142711,316708,2750970,316808,2751070,316758,2751020,142711,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# 寫入csv
result_t.to_csv("taipei_trans.csv", index=False)
result_nt.to_csv("new_taipei_trans.csv", index=False)